In [ ]:
!pip3 install datasets transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 34.7 MB/s eta 0:00:00


In [ ]:
!pip3 install wandb --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

In [ ]:
def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random, bayesian
    'metric': {
      'name': 'f1_micro',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [3e-5]
        },
        'batch_size': {
            'values': [32]
        },
        'epochs':{'value': 10},
        'dropout':{
            'values': [0.5]
        },
        'tokenizer_max_len': {'value': 40},
    }
}

sweep_defaults = {
    'learning_rate': 3e-5,
    'batch_size': 64,
    'epochs': 10,
    'dropout': 0.3,
    'tokenizer_max_len': 40
}

sweep_id = wandb.sweep(sweep_config, project='PrivBERT_Final_Run')

Create sweep with ID: 76t15mn5
Sweep URL: https://wandb.ai/sbdesai2/PrivBERT_Classification/sweeps/76t15mn5


In [ ]:
# go_emotions = load_dataset("go_emotions")
# data = go_emotions.data
# Mounting drive and importing csv's
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/Applied_ML_Project/')
TRAIN_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/train_cleaned.csv'
TEST_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/test_cleaned.csv'
VAL_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/val_cleaned.csv'
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
valid = pd.read_csv(VAL_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(train.shape, test.shape, valid.shape)

(18555, 14) (2320, 14) (2319, 14)


In [ ]:
train.dropna(inplace = True)

In [ ]:
train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...


In [ ]:
# Restructring Datasets
train_labels = np.argmax(train.iloc[:,  3:-2].values, axis=1)
test_labels = np.argmax(test.iloc[:, 3:-2].values, axis=1)
val_labels = np.argmax(valid.iloc[:, 3:-2].values, axis=1)

train['labels'] = train_labels
test['labels'] = test_labels
valid['labels'] = val_labels

train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,5
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,0
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,7
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,0
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,7


In [ ]:
# train, valid, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

In [ ]:
print(train.shape, valid.shape, test.shape) 

(18552, 15) (2319, 15) (2320, 15)


In [ ]:
train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,5
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,0
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,7
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,0
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,7


In [ ]:
for col in train.columns:
  print(col)

Unnamed: 0
index
Policy_text
Category_Data Retention
Category_Data Security
Category_Do Not Track
Category_First Party Collection/Use
Category_International and Specific Audiences
Category_Other
Category_Policy Change
Category_Third Party Sharing/Collection
Category_User Access, Edit and Deletion
Category_User Choice/Control
text
labels


In [ ]:
mapping = {
    0:"Category_Data Retention",
    1:"Category_Data Security",
    2:"Category_Do Not Track",
    3:"Category_First Party Collection/Use",
    4:"Category_International and Specific Audiences",
    5:"Category_Other",
    6:"Category_Policy Change",
    7:"Category_Third Party Sharing/Collection",
    8:"Category_User Access, Edit and Deletion",
    9:"Category_User Choice/Control",
}

n_labels = len(mapping)

In [ ]:
# def one_hot_encoder(df):
#     one_hot_encoding = []
#     for i in tqdm(range(len(df))):
#         temp = [0]*n_labels
#         label_indices = df.iloc[i]["labels"]
#         for index in label_indices:
#             temp[index] = 1
#         one_hot_encoding.append(temp)
#     return pd.DataFrame(one_hot_encoding)

In [ ]:
# train_ohe_labels = one_hot_encoder(train)
# valid_ohe_labels = one_hot_encoder(valid)
# test_ohe_labels = one_hot_encoder(test)

In [ ]:
# train_ohe_labels.shape

In [ ]:
# train = pd.concat([train, train_ohe_labels], axis=1)
# valid = pd.concat([valid, valid_ohe_labels], axis=1)
# test = pd.concat([test, test_ohe_labels], axis=1)

In [ ]:
train['labels'] = train['labels'].apply(lambda x: np.array([x]))
valid['labels'] = valid['labels'].apply(lambda x: np.array([x]))
test['labels'] = test['labels'].apply(lambda x: np.array([x]))


In [ ]:
train

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550,11964,11964,<strong> WITH WHOM DO WE SHARE THE INFORMATION...,0,0,0,0,0,0,0,1,0,0,with whom do we share the information that we ...,[7]
18551,21575,21575,We may also file your comments to improve the ...,0,0,0,1,0,0,0,0,0,0,we may also file your comments to improve the ...,[3]
18552,5390,5390,Our servers may also automatically collect inf...,0,0,0,1,0,0,0,0,0,0,our servers may also automatically collect inf...,[3]
18553,860,860,My Favorites. You may choose to register for a...,0,0,0,1,0,0,0,0,0,0,my favorites you may choose to register for a...,[3]


In [ ]:
train = train.rename(columns={'index': 'id'})
train.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)

valid = valid.rename(columns={'index': 'id'})
valid.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)

test = test.rename(columns={'index': 'id'})
test.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)
train.head()

,id,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]


In [ ]:
train = train.rename(columns = {"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })
test = test.rename(columns = {"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })
valid = valid.rename(columns={"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })


In [ ]:
train.head()

,id,0,1,2,3,4,5,6,7,8,9,text,labels
0,8395,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]


In [ ]:
train['labels'][1]

array([0])

In [ ]:
def inspect_category_wise_data(label, n=5):
    samples = train[train[label] == 1].sample(n)
    sentiment = mapping[label]
    
    print(f"{n} samples from {sentiment} sentiment: \n")
    for text in samples["text"]:
        print(text, end='\n\n')

In [ ]:
inspect_category_wise_data(4)

5 samples from Category_International and Specific Audiences sentiment: 

we will retain your or your childs information for as long as your account is active or as needed to provide you services  if you wish to cancel your or your childs account or request that we no longer use your information to provide you services contact us at support lynda com  we will retain and use your information as necessary to comply with our legal obligations  resolve disputes  and enforce our agreements  we will respond to your request to access within 30 days

for additional tips on how to help children stay safe on the internet  we recommend that you visit the following sites    www ftc gov privacy privacyinitiatives childrens html   www ftc gov bcp edu pubs consumer tech tec14 shtm   http   www onguardonline gov features feature 0002 featured info parents

what is our policy on children s privacy     the sites are not for children under 13 years of age  however  there are some areas of the sites that 

In [ ]:
class GoEmotionDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class GoEmotionClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoEmotionClassifier, self).__init__()
        self.bert = bert_model
        self.dropout1 = nn.Dropout(do_prob)
        self.out1 = nn.Linear(768, 768)
        self.dropout2 = nn.Dropout(do_prob)
        self.out2 = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout1(output_1)
        output_3 = self.out1(output_2)
        output_4 = self.dropout2(output_3)
        output_5 = self.out2(output_4)
        return output_5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mukund/privbert")

def build_dataset(tokenizer_max_len):
    train_dataset = GoEmotionDataset(train.text.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoEmotionDataset(valid.text.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    
    return train_dataset, valid_dataset

def build_dataloader(train_dataset, valid_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader

def ret_model(n_train_steps, do_prob):
  model = GoEmotionClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

In [ ]:
sample_train_dataset, _ = build_dataset(40) #40 is the tokenizer length
print(sample_train_dataset[0])
len(sample_train_dataset)

{'ids': tensor([    0, 13378, 36313,  1081,   335,  1437,  1437,  1437,   114,    47,
           74,   101,     7,    33,   110,  5636, 35432,   335,  1714,    50,
         2928,    31,    84,  8503,  1437,  2540,  1511,   201,    23,  8574,
          232,  8310,  3137,  1437,     8,    52,    40,   304,  5701,     2]), 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])}


18552

In [ ]:
bert_model = AutoModel.from_pretrained("mukund/privbert")

Some weights of the model checkpoint at mukund/privbert were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def ret_optimizer(model):
    '''
    Taken from Abhishek Thakur's Tez library example: 
    https://github.com/abhishekkrthakur/tez/blob/main/examples/text_classification/binary.py
    '''
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=wandb.config.learning_rate)
    return opt

def ret_scheduler(optimizer, num_train_steps):
    sch = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    preds1 = np.argmax(preds, axis=1)
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    labels1 = np.argmax(labels, axis=1)
    print("Preds:",preds)
    print("Labels:",labels)
    
    '''
    auc_micro_list = []
    for i in range(n_labels):
      current_pred = preds.T[i]
      current_label = labels.T[i]
      fpr_micro, tpr_micro, _ = metrics.roc_curve(current_label.T, current_pred.T)
      auc_micro = metrics.auc(fpr_micro, tpr_micro)
      auc_micro_list.append(auc_micro)
    
    return {"auc": np.array(auc_micro).mean()}
    '''
    # auc_micro_list = []
    # for i in range(n_labels):
    #   current_pred = preds.T[i]
    #   current_label = labels.T[i]
    #   fpr_micro, tpr_micro, _ = metrics.roc_curve(current_label.T, current_pred.T)
    #   auc_micro = metrics.auc(fpr_micro, tpr_micro)
    #   auc_micro_list.append(auc_micro)
    
    # return {"Micro AUC": np.array(auc_micro).mean()}
    
    #AUC Micro

    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    # #F1 Score

    # calculate micro F1 score
    micro_f1 = f1_score(labels1, preds1, average='micro')
    # print("micro f1",micro_f1)
    macro_f1 = f1_score(labels1, preds1, average='macro')
    # print("macro f1",macro_f1)
    # calculate other metrics
    accuracy = accuracy_score(labels1, preds1)
    # print("accuracy",accuracy)
    # precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels1, preds1, average='macro')
    # print("recall",recall)
    # auc_micro = roc_auc_score(labels, preds, average='micro', multi_class='ovr')
    # print("auc micro",auc_micro)
    # auc_macro = roc_auc_score(labels, preds, average='macro', multi_class='ovr')
    # print("auc macro",auc_macro)
    
    return {
        "accuracy": accuracy,
        "recall": recall,
        "auc_micro": auc_micro,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
    return train_loss
    

def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = loss_fn(outputs, targets)
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

In [ ]:
def trainer(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train_dataset, valid_dataset = build_dataset(config.tokenizer_max_len)
        train_data_loader, valid_data_loader = build_dataloader(train_dataset, valid_dataset, config.batch_size)
        print("Length of Train Dataloader: ", len(train_data_loader))
        print("Length of Valid Dataloader: ", len(valid_data_loader))
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        n_train_steps = int(len(train_dataset) / config.batch_size * 10)

        model = ret_model(n_train_steps, config.dropout)
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        model = nn.DataParallel(model)
        wandb.watch(model)
        
        n_epochs = config.epochs

        best_val_loss = 100
        best_f1_micro = 0.5
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)
          
            # auc_score = log_metrics(preds, labels)["auc_micro"]
            metrics = log_metrics(preds, labels)
            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "accuracy": metrics['accuracy'],
                "auc_micro_score": metrics['auc_micro'],
                "micro_f1": metrics['micro_f1'],
                "macro_f1": metrics['macro_f1']
            })
            print("Executing Epoch ",epoch)
            print("Average Train loss: ", avg_train_loss)
            print("Average Valid loss: ", avg_val_loss)
            print("Accuracy:",metrics['accuracy'])
            print("Micro F1-Score:",metrics['micro_f1'])
            print("Macro F1-Score:",metrics['macro_f1'])
            print("Micro AUC score: ", metrics['auc_micro'])
            curr_f1_micro = metrics['micro_f1']
            best_f1_micro = 0
            if best_f1_micro < curr_f1_micro:
                best_f1_micro = curr_f1_micro
                torch.save(model.state_dict(), "./Best_privbert_model.pt")  
                print("Model saved as current best F1-Micro is: ", best_f1_micro)
            else:
                print("Model not saved as current F1-Score is ",curr_f1_micro," which is lesser than the Best F1-Score: ",best_f1_micro)

            if  epoch ==  n_epochs:
              old_name = r"/content/Best_privbert_model.pt"
              new_name = r"/content/PrivBERT_F1_"+str(round(best_f1_micro,2))+".pt"
              os.rename(old_name, new_name)


              import shutil
              shutil.copy("/content/PrivBERT_F1_"+str(round(best_f1_micro,2))+".pt", "/content/drive/MyDrive/Applied_ML_Project/Classification Final Models/PrivBERT_F1_"+str(round(best_f1_micro,2))+".pt")
              print("Copied PrivBERT_F1_"+str(round(best_f1_micro,2))+".pt to Applied_ML_Project/Classification Final Models")


In [ ]:
wandb.agent(sweep_id, function=trainer, count=6)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: f3i9sxn6 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	learning_rate: 3e-05
wandb: 	tokenizer_max_len: 40


Length of Train Dataloader:  580
Length of Valid Dataloader:  73


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/580 [00:00<?, ?it/s]

...

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Preds: [[0.00694395 0.00689692 0.00301342 ... 0.00831312 0.00553984 0.00609235]
 [0.00663863 0.00595363 0.00283475 ... 0.00806036 0.00548304 0.00731691]
 [0.01749399 0.01439295 0.01250396 ... 0.03057515 0.04778898 0.8071271 ]
 ...
 [0.01535577 0.00936488 0.00798351 ... 0.01377963 0.02870269 0.5558072 ]
 [0.00848895 0.0146751  0.0037108  ... 0.4825669  0.00808705 0.01781113]
 [0.02107226 0.01788907 0.01678071 ... 0.03364636 0.03018173 0.84920985]]
Labels: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Accuracy: 0.7481673134971971
Micro F1-Score: 0.7481673134971972
Macro F1-Score: 0.6162343985116665
Micro AUC score:  0.9652536209358835
Average Train loss:  0.17849601147503688
Average Valid loss:  0.1262089112723196
Model saved as current best F1-Micro is:  0.7481673134971972


  0%|          | 0/290 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
# !zip -r ./SqueezeBertResults.zip ./wandb/
from google.colab import files
files.download('Best_privbert_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>